In [2]:
import torch
import tvm
from tvm import relay

# 載入 TorchScript 模型
ts_model = torch.jit.load('tc_model.pt')

# 定義輸入形狀
input_shape = (1, 4)
shape_list = [('input', input_shape)]

# 轉換為 Relay 模型
mod, params = relay.frontend.from_pytorch(ts_model, shape_list)


NotImplementedError: The following operators are not implemented: ['aten::mode']

In [5]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from hummingbird.ml import convert
import tvm
from tvm import relay


# 加载鸢尾花数据集
iris = datasets.load_iris()
X, y = iris.data, iris.target

# 训练决策树分类器
clf = RandomForestClassifier()
clf.fit(X, y)

# 使用 Hummingbird 将模型转换为 TVM 格式
hb_model = convert(clf, 'onnx', X)

# 提取 TVM 模块和参数
tvm_model = hb_model.model
tvm_mod = tvm_model['module']
tvm_params = tvm_model['params']

# 构建 Relay 模块并生成共享库
target = tvm.target.Target("llvm", host="llvm -mtriple=x86_64-linux-gnu")
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(tvm_mod, target=target, params=tvm_params)

# 导出编译后的共享库为 .so 文件
lib.export_library("output.so", cc="x86_64-linux-gnu-gcc")


============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



TypeError: 'ModelProto' object is not subscriptable

In [15]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# 加载鸢尾花数据集
iris = datasets.load_iris()
X, y = iris.data.astype(np.float32), iris.target

# 训练决策树分类器
clf = DecisionTreeClassifier()
clf.fit(X, y)

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# 定義輸入類型
initial_type = [('float_input', FloatTensorType([None, X.shape[1]]))]

# 轉換為 ONNX 模型
onnx_model = convert_sklearn(clf, initial_types=initial_type, target_opset=9)

# 指定儲存路徑
onnx_file_path = "tree.onnx"

# 將 ONNX 模型儲存為檔案
with open(onnx_file_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"ONNX 模型已儲存至 {onnx_file_path}")

from hummingbird.ml import convert

# 將 scikit-learn 模型轉換為 ONNX 格式
hb_model = convert(onnx_model, 'onnx')

# 保存轉換後的 ONNX 模型
hb_model.save('iris_logistic_regression_torch')

# 解壓縮資料夾
!unzip -o iris_logistic_regression_torch.zip -d dist

ONNX 模型已儲存至 tree.onnx
============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Model saved with digest: aa9ffdf27a4f8fc05abe252415ddadf62a486dd8
Archive:  iris_logistic_regression_torch.zip
  inflating: dist/model_type.txt     
  inflating: dist/container.pkl      
  inflating: dist/model_configuration.txt  
  inflating: dist/deploy_model.onnx  


In [1]:
import tvm
from tvm import relay
from tvm.contrib import cc, utils
from tvm.contrib import graph_executor
import onnx
import sys

# original_platform = sys.platform
# sys.platform = "linux"
# 載入 ONNX 模型
onnx_model = onnx.load("dist/deploy_model.onnx")

# 將 ONNX 模型轉換為 Relay 模型
input_name = 'float_input'  # 輸入名稱可在 ONNX 模型中確認
shape_dict = {input_name: (1, 4)}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 設置目標架構，這裡假設為通用的 CPU
# target = 'llvm'
target = tvm.target.Target("llvm", host="llvm -mtriple=x86_64-linux-gnu")
# target = tvm.target.Target("llvm", host="llvm -mtriple=aarch64-linux-gnu")
# target = tvm.target.Target("llvm", host="llvm -mtriple=x86_64-apple-darwin")
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

# 編譯輸出為 C 代碼
lib.export_library("output.so", cc="x86_64-linux-gnu-gcc")
# lib.export_library("output.so", cc="aarch64-linux-gnu-gcc")
# lib.export_library("output.so", cc="clang")

# 恢復原始平台
# sys.platform = original_platform

[09:17:02] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[09:17:02] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`
[09:17:02] /home/jovyan/project/ONNX-MLIR/tvm/src/target/llvm/llvm_instance.cc:226: Error: Using LLVM 19.1.3 with `-mcpu=apple-latest` is not valid in `-mtriple=arm64-apple-macos`, using default `-mcpu=generic`


In [ ]:
target = tvm.target.Target("llvm", host="llvm -mtriple=x86_64-linux-gnu")
# target = tvm.target.Target("llvm", host="llvm -mtriple=aarch64-linux-gnu")
# target = tvm.target.Target("llvm", host="llvm -mtriple=x86_64-apple-darwin")
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)